In [1]:
import numpy as np
import pandas as pd
import math
import copy

In [2]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [3]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [4]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
    max_distance = np.nanmax(distance)
    for i in range(cities):#
        distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [5]:
# encoding a tour
def encode(detour):
    cities = len(detour)
    master = [x for x in range(cities)]
    entour = []
    
    for i in range(cities):
        entour.append(master.index(detour[i]))
        master.remove(detour[i])
    
    return entour

In [6]:
#decoding an individual
def decode(entour):
    cities = len(entour)    
    master = [x for x in range(cities)]
    detour = []
    
    for i in range(cities):
        detour.append(master[entour[i]])
        master.remove(master[entour[i]])
    
    return detour

In [7]:
# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def tour_swap (tour, m, n):
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

In [8]:
# define function to calculate the length of the tour 
def totalcost(distance, tour):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [9]:
def two_opt (distance, tour):
    import copy
    
    best_tour = tour
    best_delta = 0
    improvement = True
    
    while improvement: 
        improvement = False
        for i in range(1,len(tour)-2):
            for j in range(i+1, len(tour)-1):
                temp_tour = tour_swap(tour, i, j)
                delta = totalcost(distance, tour[(i-1):(j+2)]) - totalcost(distance, temp_tour[(i-1):(j+2)]) #improvement = positive delta
                
                if delta > best_delta:
                    best_tour = copy.deepcopy(temp_tour)
                    best_delta = delta
                    improvement = True
    
    return (best_tour, totalcost(distance, best_tour))

In [10]:
#evaluate a tour
def evaluate(distance, tour, return_codetype, eval_two_opt = False):
    best_tour = tour
    if eval_two_opt:
        (best_tour, best_cost) = two_opt(distance, tour)
    else: #default
        best_cost = 0
        for i in range(len(tour)-1):
            best_cost += distance[tour[i]][tour[i+1]]
    
    best_cost +=  distance[tour[-1]][tour[0]] #return to starting point
    if return_codetype == 'encode':
        best_tour = encode(best_tour)
    
    return (best_tour, best_cost)

In [11]:
#format of a solution
def create_solution(distance, entour, codetype = 'permutation', create_two_opt = False):
    
    if codetype == 'encode':
        tour = decode(entour)
    elif codetype == 'permutation':
        tour = entour
    else:
        raise ValueError("Wrong codetype")
            
    
    (tour, best_cost) = evaluate(distance, tour, return_codetype = codetype, eval_two_opt = create_two_opt)
    # => return two_opt optimized tour & length if selecting two_opt option, else return original tour and its length
    # => return encode/permutation solution based on codetype
    
    solution = [best_cost, tour]
    
    return solution
    

In [12]:
# random initialize
def initialize(cities, distance, pop_size, codetype = 'permutation', init_two_opt = False):
    solution_pop = []
    entour_pop = [] #population of decision, 
                #in case of 2-opt include all original decisions, of which 2-opt optimized tour already included in solution population
    
    while len(entour_pop) < pop_size: #iterate to get pop_size candidates 
        entour = []
        
        if codetype == 'encode':    
            for i in range(cities):
                entour.append(np.random.randint(0,cities-i))  #position <= # of remaning cities
        elif codetype == 'permutation':
            entour = np.arange(cities)
            np.random.shuffle(entour)
            entour = list(entour)
        else:
            raise ValueError("Wrong codetype")            
        
        if entour not in entour_pop:
            entour_pop.append(entour)
            solution = create_solution(distance, entour, codetype, init_two_opt)
            if solution not in solution_pop: 
                solution_pop.append(solution)
    
    return solution_pop #return a population of solution candidates

In [13]:
# selection
def select(solution_pop, offspring_size, selecttype = 'fitness', tournament_size = None):
    offspring = []
    expected = np.mean([ sol[0] for sol in solution_pop])
    pop_size = len(solution_pop)
    f = [ (2*expected - solution_pop[i][0]) for i in range(pop_size)] #calculate f of each candidate
    sum_f = sum(f)

    if selecttype == 'tournament':
        #tournament size = # of solutions selected for each tournament
        if tournament_size is None: 
            tournament_size = len(solution_pop)//4
        
        for i in range(offspring_size):
            tournament = np.random.choice(np.arange(pop_size), size = tournament_size, replace = False) #random select k index
            tour_f = [f[i] for i in tournament]
            winner = tournament[np.argmax(tour_f)] #return index of winner
            offspring.append(solution_pop[winner])
            
        return offspring
    
    
    elif selecttype == 'rank': #rank selection
        f_index = [(value, idx) for (idx, value) in enumerate(f)]
        f_index = sorted(f_index, reverse = True) #sort based on value
        rank_index = [(idx, rank) for (rank, (value,idx)) in enumerate(f_index)] #get rank
        rank_index = sorted(rank_index) #sorted base on index
        rank = [r[1] for r in rank_index] #get a list of rank sorted on index
        
        eMax = 1 + np.random.random()
        eMin = 2 - eMax
        prob = [ (1/pop_size) * (eMax - (eMax - eMin)*(rank[i] - 1)/(pop_size-1)) for i in range(pop_size)] #individual prob
        
        prob_range = [ sum(prob[0:i+1]) for i in range(pop_size)] #cumulative prob of each candidate
        
    elif selecttype == 'fitness': #default selection mechanism
        prob_range = [ sum(f[0:i+1])/sum_f for i in range(pop_size)] #cumulative prob of each candidate
    
    else:
        raise ValueError("Wrong selecttype")        
    
    
    for i in range(offspring_size):
        p = np.random.random()

        for j in range(pop_size):
            if prob_range[j] >= p:
                offspring.append(solution_pop[j])
                break
        
    return offspring

In [14]:
#recombination and mutation
def recomb(cities, distance, solution_pop, offspring, recprob, muprob, 
           crossprob = 0.5, mulocprob = 0.5, crosstype = 'uniform', codetype = 'permutation', recomb_two_opt = False):

    pop = [s[1] for s in solution_pop] #set of decision

    new_solution_pop = []
    
    for k in range(len(offspring)//2): 

        #recombination
        if np.random.random() < recprob:
            child = []
            
            #crossover
            if crosstype == 'uniform': #uniform crossover
                for i in range(cities):
                    if np.random.random() < crossprob:
                        child.append(offspring[2*k][1][i]) #from offspring1
                    else: 
                        child.append(offspring[2*k+1][1][i]) #from offspring2
            
            elif crosstype == '1point': #1 point crossover
                cp=np.random.randint(1,cities-1)
                child = offspring[2*k][1][:cp] + offspring[2*k+1][1][cp:]
            
            elif crosstype == '2point': #2 point crossover
                cp1=np.random.randint(1,cities-2)
                cp2=np.random.randint(cp1+1,cities)
                child = offspring[2*k][1][:cp1] + offspring[2*k+1][1][cp1:cp2] + offspring[2*k][1][cp2:]
            else:
                raise ValueError("Wrong selecttype")
            
            
            #mutation: 2 different types of mutation for each coding style
            if np.random.random() < muprob:
                if codetype == 'permutation':
                    (loc1, loc2) = np.random.choice(child, size = 2, replace = False)
                    temp = child[loc1]
                    child[loc1] = child[loc2]
                    child[loc2] = temp
                    
                elif codetype == 'encode':
                    for i in range(cities):
                        if np.random.random() < mulocprob:
                            child[i] = min(cities-1-i,child[i]+1)
                        else: child[i] = max(0,child[i]-1)
                            
                else:
                    raise ValueError("Wrong codetype")


            #add to population
            if child not in pop:
                pop.append(child)
                new_solution = create_solution(distance, child, codetype, recomb_two_opt)
                if new_solution not in solution_pop:
                    new_solution_pop.append(new_solution)
            
    return new_solution_pop

In [15]:
# replacement
def replace(solution_pop, new_solution_pop, pop_size, reptype = 'best', selecttype = None):
    pop = []
    
    if reptype == 'complete':
        return new_solution_pop
    
    elif reptype == 'best':
        total_pop = solution_pop + new_solution_pop
        sortedpop = sorted(total_pop) #sorted by cost of a solution
        pop = sortedpop[0:pop_size]
        
    elif reptype == 'select':
        total_pop = solution_pop + new_solution_pop
        pop = select(total_pop, pop_size, selecttype)
    
    return pop 

In [16]:
#main algorithm
def GA_TSP (cities, distance, 
            pop_size, offspring_size, recprob, crossprob, muprob, mulocprob,
            codetype, two_opt_select, selecttype, crosstype, reptype, 
            max_iter = None, TimeLimit = None, obj_length = None):

    from time import process_time 
    start_time = process_time()      #computational time benchmarking
           
    
    solution_pop = initialize(cities, distance, pop_size, codetype = codetype, init_two_opt = two_opt_select)
    
        
    terminated = False
    it = 0 #iteration
    
    while not terminated: #termination condition    
        it += 1

        offspring = select(solution_pop, offspring_size, selecttype = selecttype)

        new_solution_pop = recomb(cities, distance, solution_pop, offspring,
                              recprob = recprob, muprob = muprob, crossprob = crossprob, mulocprob = mulocprob, 
                              crosstype = crosstype, codetype = codetype, recomb_two_opt = two_opt_select)

        solution_pop = replace(solution_pop, new_solution_pop, pop_size, reptype = reptype, selecttype = selecttype)


        sorted_solution_pop = sorted(solution_pop) #sorted by cost of a solution
        #print('Objective values of current population', [s[0] for s in sorted_solution_pop])
        
        runtime = process_time() - start_time
        #check termination condition
        if TimeLimit is not None:
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True
                   
        elif obj_length is not None:
            if (sorted_solution_pop[0][0] <= obj_length):
                print('Reach objective length')
                terminated = True              
                
        elif max_iter is not None:
            if (it >= max_iter):
                print('Reach max number of iterations')
                terminated = True
                
        else:
            if (it >= 10): # default max number of iterations
                print('Reach default max number of iterations')
                terminated = True
                           
    print('Running time: ', runtime)
    best_solution = sorted_solution_pop[0]         #get the best candidate
    print('Best objective value', best_solution[0])
    #print('Decision', best_solution[1])
    
    return (best_solution[0], runtime)

In [17]:
#prepare dataset

In [18]:
# 120 cities
cities0 = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance0=di.values

for i in range(cities0):
    distance0[i,i] = 10000

In [19]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [20]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

<a id='set_params'></a>
**Setting default params**

In [21]:
max_iter_param = 500

pop_size_param = 500

offspring_size_param = 1000

recprob_param = 0.7
crossprob_param = 0.5

muprob_param = 0.5
mulocprob_param = 0.3

In [22]:
codetype_param = 'permutation' #encode
two_opt_select_param = False #True
selecttype_param = 'rank' #fitness, tournament
crosstype_param = 'uniform' #1point, 2point
reptype_param = 'best' #complete, select

In [23]:
(result, time) = GA_TSP (cities0, distance0, 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)

Reach max number of iterations
Running time:  35.984375
Best objective value 7581


## Comparison

**Problem specific parameters**  
• Solution representation + mutation mechanism => encode (move to left) vs permutation (swap 2 points)  
• Evaluation of solutions => 2-opt vs not 2-opt  
• Design of genetic operators recombination and crossover => uniform vs 1-point crossover vs 2-point crossover  

**Generic parameters**  
• Population size  
• Number of newly generated solutions in each iteration  
• Selection mechanism => fitness vs rank vs tournament  
• Recombination and mutation probabilities  
• Replacement strategy => best out of (N+M) vs complete replacement M=N vs selection from (N+M)
• Termination criterion => max number of iterations/TimeLimit vs objective length  


In [24]:
dataset = ((cities0, distance0), (cities1, distance1), (cities2, distance2))
index = ['Dataset 1', 'Dataset 2', 'Dataset 3']

**Notation**  
Dataset 1 = Example with 120 cities (best obj value = 6942)  
Dataset 2 = Example with 152 cities (best obj value = 73682)  
Dataset 3 = Example with 264 cities (best obj value = 49135)

### Problem specific

#### Solution representation + mutation mechanism  
Run 2 combinations of (representation type with mutation mechanism): 1) encode by sequence position and mutation by moving to left and 2) permutation coding and mutation by swap 2 cities with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. The test is run 2 times and the average result is considered for evaluation.

In [25]:
value = pd.DataFrame(index = index, columns = ['encode', 'permutation'])
codetype_set = ['encode', 'permutation']
obj_set = np.zeros([2,3,2])
time_set = np.zeros([2,3,2])

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        for j in range(2): #codetype
            result = GA_TSP (dataset[i][0], dataset[i][1], 
                             pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                             crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                             codetype = codetype_set[j], two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                             crosstype = crosstype_param, reptype = reptype_param, 
                             max_iter = max_iter_param, TimeLimit = None, obj_length = None)
            obj_set[k,i,j] = result[0]
            time_set[k,i,j] = result[1]

for i in range(3): #dataset 
    for j in range(2): #codetype
        obj_mean = np.mean([obj_set[k,i,j] for k in range(2)])
        time_mean = np.mean([time_set[k,i,j] for k in range(2)])
        value.iloc[i,j] = (obj_mean, time_mean)
        

Reach max number of iterations
Running time:  73.390625
Best objective value 41799
Reach max number of iterations
Running time:  38.4375
Best objective value 8508
Reach max number of iterations
Running time:  80.78125
Best objective value 237523.1939640085
Reach max number of iterations
Running time:  43.234375
Best objective value 102643.07945942097
Reach max number of iterations
Running time:  191.203125
Best objective value 252891.15478419527
Reach max number of iterations
Running time:  66.453125
Best objective value 209431.31664427608
Reach max number of iterations
Running time:  75.140625
Best objective value 42417
Reach max number of iterations
Running time:  42.90625
Best objective value 7733
Reach max number of iterations
Running time:  81.78125
Best objective value 251056.27824250056
Reach max number of iterations
Running time:  44.375
Best objective value 108819.53793531988
Reach max number of iterations
Running time:  184.109375
Best objective value 267621.1707386768
Reach 

In [26]:
value1=value
print(value)

                                    encode                       permutation
Dataset 1             (42108.0, 74.265625)               (8120.5, 40.671875)
Dataset 2   (244289.73610325455, 81.28125)  (105731.30869737043, 43.8046875)
Dataset 3  (260256.16276143602, 187.65625)  (224172.77516969072, 69.1328125)


#### Comment:   
Overall, with default setting, the performance of encode and permutation is not too much different. While the encode get slightly better results in dataset 1 and 3, the permutation is much better in dataset 2. Thee running time is nearly same between 2 approaches.

#### Evaluation of solutions   
Run 2 evaluation strategy: 1) with 2-opt (i.e. firstly find the best possible tour from original tour and build solution based on this best tour) and 2) without 2-opt (i.e. directly use the length of the tour). The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. The test is run 2 times and the average result is considered for evaluation.

In [28]:
value = pd.DataFrame(index = index, columns = ['with 2-opt', 'without 2-opt'])
two_opt_select_set = [True, False]
obj_set = np.zeros([2,3,2])
time_set = np.zeros([2,3,2])

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        for j in range(2): #2-opt option
            result = GA_TSP (dataset[i][0], dataset[i][1], 
                             pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                             crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                             codetype = codetype_param, two_opt_select = two_opt_select_set[j], selecttype = selecttype_param, 
                             crosstype = crosstype_param, reptype = reptype_param, 
                             max_iter = None, TimeLimit = 100, obj_length = None)
            obj_set[k,i,j] = result[0]
            time_set[k,i,j] = result[1]

for i in range(3): #dataset 
    for j in range(2): #codetype
        obj_mean = np.mean([obj_set[k,i,j] for k in range(2)])
        time_mean = np.mean([time_set[k,i,j] for k in range(2)])
        value.iloc[i,j] = (obj_mean, time_mean)

KeyboardInterrupt: 

In [ ]:
value2=value
print(value)

#### Comment:   
Overall, with default setting, the performance of encode and permutation is not too much different. While the encode get slightly better results in dataset 1 and 3, the permutation is much better in dataset 2. Thee running time is nearly same between 2 approaches.

#### Design of genetic operators recombination and crossover    
Run 3 crossover strategy: 1) uniform ; 2) 1-point crossover and 3) 2-point crossover. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. The test is run 2 times and the average result is considered for evaluation.

In [29]:
value = pd.DataFrame(index = index, columns = ['uniform', '1-point crossover', '2-point crossover'])
crosstype_set = ['uniform', '1point', '2point']
obj_set = np.zeros([2,3,3])
time_set = np.zeros([2,3,3])

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        for j in range(3): #crossover options
            result = GA_TSP (dataset[i][0], dataset[i][1], 
                             pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                             crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                             codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                             crosstype = crosstype_set[j], reptype = reptype_param, 
                             max_iter = max_iter_param, TimeLimit = None, obj_length = None)
            obj_set[k,i,j] = result[0]
            time_set[k,i,j] = result[1]

for i in range(3): #dataset 
    for j in range(3): #codetype
        obj_mean = np.mean([obj_set[k,i,j] for k in range(2)])
        time_mean = np.mean([time_set[k,i,j] for k in range(2)])
        value.iloc[i,j] = (obj_mean, time_mean)

Reach max number of iterations
Running time:  40.390625
Best objective value 8327
Reach max number of iterations
Running time:  24.296875
Best objective value 12934
Reach max number of iterations
Running time:  24.203125
Best objective value 11502
Reach max number of iterations
Running time:  47.53125
Best objective value 97208.88626424356
Reach max number of iterations
Running time:  28.390625
Best objective value 238886.9876483038
Reach max number of iterations
Running time:  30.03125
Best objective value 228227.13348541243
Reach max number of iterations
Running time:  63.984375
Best objective value 220927.76703870026
Reach max number of iterations
Running time:  40.796875
Best objective value 305241.1228981491
Reach max number of iterations
Running time:  40.515625
Best objective value 247935.13584320722
Reach max number of iterations
Running time:  34.234375
Best objective value 8047
Reach max number of iterations
Running time:  25.0
Best objective value 12821
Reach max number of i

In [30]:
value3=value
print(value)

                                  uniform                1-point crossover  \
Dataset 1               (8187.0, 37.3125)            (12877.5, 24.6484375)   
Dataset 2     (95316.8653305158, 45.5625)  (224870.00194626336, 28.484375)   
Dataset 3  (226837.4277794746, 64.546875)  (299390.68652711494, 40.515625)   

                         2-point crossover  
Dataset 1            (12355.5, 24.5390625)  
Dataset 2  (219866.91144681952, 30.421875)  
Dataset 3  (265906.7111827731, 40.5546875)  


#### Comment:   
Overall, with default setting, the performance of encode and permutation is not too much different. While the encode get slightly better results in dataset 1 and 3, the permutation is much better in dataset 2. Thee running time is nearly same between 2 approaches.

In [37]:
value = pd.DataFrame(index = index, columns = ['pop_size = 100', 'pop_size = 250', 'pop_size = 500', 'pop_size = 1000'])
pop_size_set = [100, 250, 500, 1000]
offspring_size_set = [200, 500, 1000, 2000]

for i in range(3): #dataset 
    for j in range(4): #population size
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_set[j], offspring_size = offspring_size_set[j], recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result



Reach max number of iterations
Running time:  4.96875
Best objective value 12830
Reach max number of iterations
Running time:  13.828125
Best objective value 11147
Reach max number of iterations
Running time:  33.3125
Best objective value 7333
Reach max number of iterations
Running time:  97.03125
Best objective value 5449
Reach max number of iterations
Running time:  5.796875
Best objective value 203898.38820381404
Reach max number of iterations
Running time:  17.3125
Best objective value 122309.68214942455
Reach max number of iterations
Running time:  41.9375
Best objective value 105635.07918627733
Reach max number of iterations
Running time:  122.3125
Best objective value 66861.66396818432
Reach max number of iterations
Running time:  9.84375
Best objective value 377657.807845032
Reach max number of iterations
Running time:  27.03125
Best objective value 264026.7513982314
Reach max number of iterations
Running time:  64.28125
Best objective value 163344.7028506485
Reach max number o

In [38]:
value4=value
print(value)

                           pop_size = 100                 pop_size = 250  \
Dataset 1                (12830, 4.96875)             (11147, 13.828125)   
Dataset 2  (203898.38820381404, 5.796875)  (122309.68214942455, 17.3125)   
Dataset 3     (377657.807845032, 9.84375)  (264026.7513982314, 27.03125)   

                          pop_size = 500                 pop_size = 1000  
Dataset 1                (7333, 33.3125)                (5449, 97.03125)  
Dataset 2  (105635.07918627733, 41.9375)   (66861.66396818432, 122.3125)  
Dataset 3  (163344.7028506485, 64.28125)  (177330.8741864757, 162.78125)  


In [35]:
value = pd.DataFrame(index = index, columns = ['offspring_size = 500', 'offspring_size = 1000', 'offspring_size = 2000'])
offspring_size_set = [500, 1000, 2000]

for i in range(3): #dataset 
    for j in range(3): #offspring size
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_set[j], recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

Reach max number of iterations
Running time:  19.125
Best objective value 7278
Reach max number of iterations
Running time:  41.421875
Best objective value 8243
Reach max number of iterations
Running time:  84.03125
Best objective value 6281
Reach max number of iterations
Running time:  22.203125
Best objective value 86486.43714728934
Reach max number of iterations
Running time:  50.515625
Best objective value 105855.84553272101
Reach max number of iterations
Running time:  98.1875
Best objective value 91963.62540538292
Reach max number of iterations
Running time:  35.125
Best objective value 216732.38161854277
Reach max number of iterations
Running time:  71.5
Best objective value 193307.23980931123
Reach max number of iterations
Running time:  143.5625
Best objective value 211811.93086329693


In [36]:
value5=value
print(value)

                     offspring_size = 500            offspring_size = 1000  \
Dataset 1                  (7278, 19.125)                (8243, 41.421875)   
Dataset 2  (86486.43714728934, 22.203125)  (105855.84553272101, 50.515625)   
Dataset 3    (216732.38161854277, 35.125)       (193307.23980931123, 71.5)   

                    offspring_size = 2000  
Dataset 1                (6281, 84.03125)  
Dataset 2    (91963.62540538292, 98.1875)  
Dataset 3  (211811.93086329693, 143.5625)  


In [39]:
value = pd.DataFrame(index = index, columns = ['fitness', 'rank', 'tournament'])
selecttype_set = ['fitness', 'rank', 'tournament']

for i in range(3): #dataset 
    for j in range(3): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_set[j], 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

Reach max number of iterations
Running time:  44.53125
Best objective value 5895
Reach max number of iterations
Running time:  35.046875
Best objective value 7854
Reach max number of iterations
Running time:  49.65625
Best objective value 16551
Reach max number of iterations
Running time:  52.25
Best objective value 94163.06077715915
Reach max number of iterations
Running time:  42.703125
Best objective value 102598.29966595452
Reach max number of iterations
Running time:  55.71875
Best objective value 342615.8221938928
Reach max number of iterations
Running time:  73.6875
Best objective value 203697.8103565038
Reach max number of iterations
Running time:  62.78125
Best objective value 204370.0641474295
Reach max number of iterations
Running time:  73.3125
Best objective value 368736.5320559888


In [40]:
value6=value
print(value)

                                fitness                             rank  \
Dataset 1              (5895, 44.53125)                (7854, 35.046875)   
Dataset 2    (94163.06077715915, 52.25)  (102598.29966595452, 42.703125)   
Dataset 3  (203697.8103565038, 73.6875)    (204370.0641474295, 62.78125)   

                              tournament  
Dataset 1              (16551, 49.65625)  
Dataset 2  (342615.8221938928, 55.71875)  
Dataset 3   (368736.5320559888, 73.3125)  


In [41]:
value = pd.DataFrame(index = index, columns = ['recombination probabiliy = 0', '= 0.4', '= 0.7', '= 1'])
recprob_set = [0, 0.4, 0.7, 1]

for i in range(3): #dataset 
    for j in range(4): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_set[j], 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

Reach max number of iterations
Running time:  5.28125
Best objective value 44607
Reach max number of iterations
Running time:  23.0625
Best objective value 8246
Reach max number of iterations
Running time:  36.78125
Best objective value 8294
Reach max number of iterations
Running time:  46.828125
Best objective value 6857
Reach max number of iterations
Running time:  5.15625
Best objective value 937638.4894699118
Reach max number of iterations
Running time:  27.015625
Best objective value 99509.96800784777
Reach max number of iterations
Running time:  44.828125
Best objective value 101208.56430943559
Reach max number of iterations
Running time:  60.5625
Best objective value 96818.41957847042
Reach max number of iterations
Running time:  5.3125
Best objective value 979118.8983544551
Reach max number of iterations
Running time:  39.640625
Best objective value 245076.9369597952
Reach max number of iterations
Running time:  65.359375
Best objective value 248225.73972138856
Reach max number

In [42]:
value7=value
print(value)

           recombination probabiliy = 0                           = 0.4  \
Dataset 1              (44607, 5.28125)                 (8246, 23.0625)   
Dataset 2  (937638.4894699118, 5.15625)  (99509.96800784777, 27.015625)   
Dataset 3   (979118.8983544551, 5.3125)  (245076.9369597952, 39.640625)   

                                     = 0.7                             = 1  
Dataset 1                 (8294, 36.78125)               (6857, 46.828125)  
Dataset 2  (101208.56430943559, 44.828125)    (96818.41957847042, 60.5625)  
Dataset 3  (248225.73972138856, 65.359375)  (201006.7584686698, 90.359375)  


In [43]:
value = pd.DataFrame(index = index, columns = ['mutation probabiliy = 0', '= 0.25', '= 0.5', '= 0.75', '= 1'])
muprob_set = [0, 0.25, 0.5, 0.75, 1]

for i in range(3): #dataset 
    for j in range(5): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_set[j], mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

Reach max number of iterations
Running time:  24.984375
Best objective value 10279
Reach max number of iterations
Running time:  32.75
Best objective value 9509
Reach max number of iterations
Running time:  36.125
Best objective value 7356
Reach max number of iterations
Running time:  38.109375
Best objective value 6900
Reach max number of iterations
Running time:  42.890625
Best objective value 7713
Reach max number of iterations
Running time:  29.609375
Best objective value 138398.13971305508
Reach max number of iterations
Running time:  38.84375
Best objective value 113074.25770189997
Reach max number of iterations
Running time:  50.390625
Best objective value 108105.41583571461
Reach max number of iterations
Running time:  46.359375
Best objective value 94049.16894349063
Reach max number of iterations
Running time:  47.359375
Best objective value 96106.96820296111
Reach max number of iterations
Running time:  49.453125
Best objective value 250094.53206539058
Reach max number of ite

In [44]:
value8=value
print(value)

                   mutation probabiliy = 0                          = 0.25  \
Dataset 1               (10279, 24.984375)                   (9509, 32.75)   
Dataset 2  (138398.13971305508, 29.609375)  (113074.25770189997, 38.84375)   
Dataset 3  (250094.53206539058, 49.453125)  (218614.5833315186, 60.140625)   

                                     = 0.5                          = 0.75  \
Dataset 1                   (7356, 36.125)               (6900, 38.109375)   
Dataset 2  (108105.41583571461, 50.390625)  (94049.16894349063, 46.359375)   
Dataset 3    (218696.71515966186, 64.0625)  (213822.93844656512, 65.90625)   

                                      = 1  
Dataset 1               (7713, 42.890625)  
Dataset 2  (96106.96820296111, 47.359375)  
Dataset 3  (217685.41608983523, 67.78125)  


In [46]:
value = pd.DataFrame(index = index, columns = ['complete replacement', 'best of pool', 'selection from pool'])
reptype_set = ['complete', 'best', 'select']

for i in range(3): #dataset 
    for j in range(3): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_set[j], 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

Reach max number of iterations
Running time:  32.90625
Best objective value 10807
Reach max number of iterations
Running time:  34.890625
Best objective value 8350
Reach max number of iterations
Running time:  43.4375
Best objective value 12371
Reach max number of iterations
Running time:  39.96875
Best objective value 151712.57584071887
Reach max number of iterations
Running time:  44.59375
Best objective value 106923.46153649916
Reach max number of iterations
Running time:  50.96875
Best objective value 165919.41631314103
Reach max number of iterations
Running time:  58.296875
Best objective value 320608.88046585576
Reach max number of iterations
Running time:  63.546875
Best objective value 220334.02061444006
Reach max number of iterations
Running time:  69.65625
Best objective value 275858.60022264


In [47]:
value9=value
print(value)

                      complete replacement                     best of pool  \
Dataset 1                (10807, 32.90625)                (8350, 34.890625)   
Dataset 2   (151712.57584071887, 39.96875)   (106923.46153649916, 44.59375)   
Dataset 3  (320608.88046585576, 58.296875)  (220334.02061444006, 63.546875)   

                      selection from pool  
Dataset 1                (12371, 43.4375)  
Dataset 2  (165919.41631314103, 50.96875)  
Dataset 3     (275858.60022264, 69.65625)  


In [50]:
value = pd.DataFrame(index = index, columns = ['time limit', 'desired value'])
TimeLimit_set = [60, 120, 120]
for i in range(3): #dataset 
    result = GA_TSP (dataset[i][0], dataset[i][1], 
                     pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                     crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                     codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                     crosstype = crosstype_param, reptype = reptype_param, 
                     max_iter = None, TimeLimit = TimeLimit_set[i], obj_length = None)
    value.iloc[i,1] = result



obj_length_set = [8400, 120000, 250000]
#desired objective value
for i in range(3): #dataset 
    result = GA_TSP (dataset[i][0], dataset[i][1], 
                     pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                     crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                     codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                     crosstype = crosstype_param, reptype = reptype_param, 
                     max_iter = None, TimeLimit = None, obj_length = obj_length_set[i])
    value.iloc[i,1] = result

Reach TimeLimit
Running time:  60.046875
Best objective value 7515
Reach TimeLimit
Running time:  120.03125
Best objective value 99275.26425701236
Reach TimeLimit
Running time:  120.078125
Best objective value 210292.38129297947
Reach objective length
Running time:  15.34375
Best objective value 8397
Reach objective length
Running time:  14.453125
Best objective value 119651.63615858275
Reach objective length
Running time:  37.59375
Best objective value 249526.04267160967


In [51]:
value10=value
print(value)

          time limit                    desired value
Dataset 1        NaN                 (8397, 15.34375)
Dataset 2        NaN  (119651.63615858275, 14.453125)
Dataset 3        NaN   (249526.04267160967, 37.59375)
